<a href="https://colab.research.google.com/github/deeplearningexplore/deeplearning1/blob/master/wildcatsclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !git init
# !git config --global user.email “<email>”
# !git config --global user.name “<username>”
# !git remote add origin https://<username>:<password>@github.com/deeplearningexplore/deeplearning1.git
# !git remote set-url origin https://<username>:<password>@github.com/deeplearningexplore/deeplearning1.git
# !git add .
# !git commit -m "hii"
# !git pull origin master --allow-unrelated-histories

Reinitialized existing Git repository in /content/.git/
